In [1]:
from selenium import webdriver
from core import MySQLcompatible
from bs4 import BeautifulSoup
import requests
import re
import utils
import json
import datetime

insert into ignore

In [2]:
tabelas = {}

tabelas['jogos_uni'] = (
"CREATE TABLE IF NOT EXISTS `jogos_uni` ("
"  `id` int(11) NOT NULL,"
"  `titulo` varchar(250) NOT NULL,"
"  `data` datetime NOT NULL,"
"  `slugLiga` varchar(120) DEFAULT NULL,"
"  `pais` varchar(100) DEFAULT NULL,"
"  `liga` varchar(100) DEFAULT NULL,"
"  `status` int(11) NOT NULL,"
"  `posicao` int(11) NOT NULL,"
"   PRIMARY KEY(`id`)"    
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

tabelas['modal_uni'] = (
"CREATE TABLE IF NOT EXISTS `modal_uni` ("
"  `id` int(11) NOT NULL,"
"  `jogo_id` int(11) NOT NULL,"
"  `odd_id` int(11) NOT NULL,"
"  `cat_id` int(11) NOT NULL,"
"  `categoria` varchar(250) NOT NULL,"
"  `propriedade` varchar(250) NOT NULL,"
"  `valor` decimal(8,2) NOT NULL DEFAULT '0.00',"
"  `status` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get("https://bets93.net/")
browser.find_element_by_xpath("//div[@class='lateral']/div/ul").click() #clica no botao "todos"

In [4]:
pattern_campeonato = re.compile(r"c_visivel")
pattern_jogo = re.compile(r"j_visivel_")

In [5]:
soup = BeautifulSoup(browser.page_source, "html.parser")
browser.quit()

In [6]:
tabela_jogos = soup.find(class_="jogos")
jogos = tabela_jogos.findAll("div",recursive=False)

In [7]:
headers = {
    'sec-fetch-mode': 'cors',
    'cookie': '__cfduid=d286a6b04f21dc52264db8885f48b8f971570143846',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://bets93.net/',
    'authority': 'bets93.net',
    'x-requested-with': 'XMLHttpRequest',
    'sec-fetch-site': 'same-origin',
}

In [8]:
id_jogos = []
for jogo in jogos:
    attr = jogo.get("id")
    if pattern_campeonato.match(attr):
        NOME_CAMP = jogo.find(class_="camp").text
        slugLiga = NOME_CAMP.lower().replace(' ','-')
        slugLiga = re.sub(pattern=r'(^-|-$)',repl='', string=slugLiga)
        pais = NOME_CAMP.split()[0]
        liga = NOME_CAMP.split()[1:]
        liga = ' '.join(liga)
        status = 1
        posicao = 1
    elif pattern_jogo.match(attr):
        id_jogo = int(attr.split('_')[-1])
        titulo = jogo.find(class_="times fundojogos").text.split()[:-3]
        titulo = ' '.join(titulo)        
        data_hora = jogo.find(class_="datahora").text
        
        ano = int(data_hora.split()[0].split('/')[2])
        mes = int(data_hora.split()[0].split('/')[1])
        dia = int(data_hora.split()[0].split('/')[0])
        horas = int(data_hora.split()[-1].split(':')[0])
        minutos = int(data_hora.split()[-1].split(':')[1])
        
        data_hora = str(datetime.datetime(ano,mes,dia,horas,minutos))
        
        params = (('id_jogo', id_jogo),)

        response = requests.get('https://bets93.net/api.php', headers=headers, params=params)        
        
        tabela2 = []  
        for dicionario in response.json():
            tabela2.append((dicionario['id_tipo_modalidade'], dicionario['id_modalidade'], dicionario['id_odd'], dicionario['odd']))        
        tabela2 = sorted(tabela2, key=lambda x:int(x[0])/1)
        
        categoria = []        
        propriedade = []
        odd_id = []
        valor = []

        for id_tipo_modalidade, id_modalidade, id_odd, odd in tabela2:
            categoria.append(int(id_tipo_modalidade))
            propriedade.append(int(id_modalidade))
            odd_id.append(int(id_odd))
            valor.append(float(odd))
        
        jogos_uni = {
            'id_jogo':id_jogo, 'titulo': titulo, 'data_hora': data_hora, 'slugLiga': slugLiga,
            'pais':pais, 'liga': liga, 'status': status, 'posicao': posicao
        }
        modal_uni = {
            'jogo_id': id_jogo, 'odd_id':odd_id, 'cat_id':1, 'categoria':categoria,
            'propriedade':propriedade, 'valor':valor, 'id_modal':1, 'status':1
        }
        break

In [9]:
with MySQLcompatible("daniel","123456789") as database:
    db = database[0]
    cursor = database[1]
    utils.create_database(cursor,"bets93")
    utils.create_table(db, cursor, tabelas)
#     utils.insert_into_jogos_uni(db, cursor, jogos_uni)
    utils.insert_into_modal_uni(db, cursor, modal_uni)    

ProgrammingError: Failed processing pyformat-parameters; Python 'list' cannot be converted to a MySQL type